In [1]:
import pickle
import pandas as pd
import matplotlib
import os
import re

import collections
import datetime
import time
 
import geopandas as gpd

import numpy as np

import rasterio

from difflib import get_close_matches

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

import dask.dataframe as dd

import gc


/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Sample

In [2]:
def getData(weatherType,yearRange1,yearRange2):

    if weatherType == "Tmax":
        weatherVar = "temperature"
    else: 
        weatherVar = "precipitation"
    
    year = yearRange1
    filename = "../../../../../../../Volumes/backup2/dissData/prism/zipcode" + weatherType + str(year) + ".csv"
    data = dd.read_csv(filename, assume_missing=True)[['ZIP','date',weatherVar]]
    # data = data[~(data[weatherVar].isna().compute())]

    years = range(yearRange1 + 1,yearRange2 + 1)
    for year in years:
        filename = "../../../../../../../Volumes/backup2/dissData/prism/zipcode" + weatherType + str(year) + ".csv"
        tempData = dd.read_csv(filename, assume_missing=True)[['ZIP','date',weatherVar]]
        # tempData = tempData[~(tempData[weatherVar].isna().compute())]
        data = data.append(tempData)

    data = data[~(data[weatherVar].isna())].compute()
    
    return(data)


In [ ]:
numberQuants    = 20

quantiles       = np.arange(0.0, 1.05, 1/numberQuants)

quant_labels   = ['quant_' + str(n)[0:4] for n in quantiles]

print(quant_labels)

In [ ]:
len(quantiles)

## Precipitation

In [3]:
start = time.time()
weatherType = "Precip" # Tmax
data = getData(weatherType,1981,2009)

### quantiles

In [ ]:
precipQuants = data.precipitation.quantile(q = quantiles)

print(time.time() - start)

In [ ]:
filename = '../../../../../../../Volumes/backup2/dissData/prism/precipQuants_1981.2009.txt'
'''with open(filename, 'wb') as handle:
    pickle.dump(precipQuants, handle)'''
    
with open(filename, 'w') as f:
    f.write(str(precipQuants.values))
    
del data
gc.collect()

### averages

In [4]:
data.head()

,ZIP,date,precipitation
142,1001.0,19810101.0,0.0
143,1002.0,19810101.0,0.0
144,1003.0,19810101.0,0.0
145,1005.0,19810101.0,0.0
146,1007.0,19810101.0,0.0


## Temperature

In [ ]:
start = time.time()
weatherType = "Tmax"
data = getData(weatherType, 1981, 2009)

tmaxQuants = data.temperature.quantile(q = quantiles) # following is no longer nec bc we call compute for nas: .compute()

print(time.time() - start)

In [ ]:
filename = '../../../../../../../Volumes/backup2/dissData/prism/tmaxQuants_1981.2009.pkl'
with open(filename, 'wb') as handle:
    pickle.dump(tmaxQuants, handle)
    
del data
gc.collect()

In [ ]:
print(tmaxQuants)

### averages

## Now go through the data from 2010 - 2019 and find time spent in each bin

In [ ]:
filename = '../../../../../../../Volumes/backup2/dissData/prism/tmaxQuants_1981.2009.pkl'
'''with open(filename, 'wb') as handle:
    pickle.dump(tmaxQuants, handle)
    '''
tmaxQuants = pd.read_pickle(filename)
tmaxQuants

In [ ]:
filename = '../../../../../../../Volumes/backup2/dissData/prism/precipQuants_1981.2009.txt'
precipQuants = open(filename, "r").read()
precipQuants

In [ ]:
quant_labels

In [ ]:
def findQuarterlyQuants(weatherType, quartileList, df, quant_labels):
    
    weatherCut = weatherType + "Cut"
    weatherCutLabels = weatherCut + 'labels'
    
    df[weatherCutLabels] = pd.Series(pd.cut(df[weatherType],
                                      bins = list(quartileList),
                                     labels = quant_labels[1:], duplicates = 'drop'))
    
    df[weatherCut] = pd.Series(pd.cut(df[weatherType],
                                      bins = list(quartileList)))
    
    df['month'] = df.date.astype(str).str.slice(4,6)

    df['quarter'] = 'q1'

    df.loc[df['month'].isin(['04','05','06']), 'quarter'] = 'q2'
    df.loc[df['month'].isin(['07','08','09']), 'quarter'] = 'q3'
    df.loc[df['month'].isin(['10','11','12']), 'quarter'] = 'q4'

    df['yearQuarter'] = df.date.astype(str).str.slice(0,4) + df.quarter

    occurrences = weatherType + "Occurences" 

    summaryDF = df.groupby(['ZIP','yearQuarter',weatherCutLabels]).size().reset_index()
    summaryDF.columns = ['zip','yearQuarter',weatherCutLabels,occurrences]
    
    '''
    summaryDF = df.groupby(['ZIP','yearQuarter',weatherCut]).size().reset_index()
    summaryDF.columns = ['zip','yearQuarter',weatherCut,occurrences]
    '''
    
    return(summaryDF)

In [ ]:
start = time.time()
recentDecadeTemps = getData("Tmax",2009,2009)
print(time.time() - start)

tempQuarterlyBins = findQuarterlyQuants(weatherType = 'temperature', quartileList = tmaxQuants, df = recentDecadeTemps)
print(tempQuarterlyBins.head())

print(time.time() - start)

tempQuarterlyBinsPivot = tempQuarterlyBins.pivot(index=['zip','yearQuarter'], columns='temperatureCutlabels', 
                            values='temperatureOccurences').reset_index(drop = True).reset_index().rename_axis(None, axis=1)

tempQuarterlyBinsPivot.to_csv("../../../../../../../Volumes/backup2/dissData/prism/tempQuarterlyBins_2009.csv")

In [ ]:
del tempQuarterlyBins
del recentDecadeTemps

gc.collect()

Many of the precipitation quartiles are duplicated. Let's make sure we just have the unique values.

In [ ]:
unique_indices = [list(precipQuants).index(i) for i in precipQuants.unique()]
quant_labels_subset = [quant_labels[i] for i in unique_indices]
precipQuants_subset = [list(precipQuants)[i] for i in unique_indices]

print(precipQuants_subset,quant_labels_subset)

In [ ]:
quant_labels_subset

In [ ]:
start = time.time()
recentDecadePrecip = getData("Precip",2009,2009)
print(time.time() - start)

In [ ]:
precipQuarterlyBins = findQuarterlyQuants(weatherType = 'precipitation', quartileList = precipQuants_subset, 
                                          df = recentDecadePrecip, quant_labels = quant_labels_subset)
print(precipQuarterlyBins.head())

print(time.time() - start)

precipQuarterlyBinsPivot = precipQuarterlyBins.pivot(index=['zip','yearQuarter'], 
        columns='precipitationCutlabels', values='precipitationOccurences').reset_index().\
        rename_axis(None, axis=1)

precipQuarterlyBinsPivot.columns.values[2:] = 'precip_' + precipQuarterlyBinsPivot.columns.values[2:]



precipQuarterlyBinsPivot.to_csv("../../../../../../../Volumes/backup2/dissData/prism/precipQuarterlyBins_2009.csv")

In [ ]:
test = pd.read_csv("../../../../../../../Volumes/backup2/dissData/prism/tempQuarterlyBins_2010.2019.csv").drop(columns = {"Unnamed: 0"})
test = test.rename_axis(None, axis = 1)

In [ ]:
test.head()

In [ ]:
test.columns.values[2:] = 'tmax_' + test.columns.values[2:]


In [ ]:
test.to_csv("../../../../../../../Volumes/backup2/dissData/prism/tempQuarterlyBins_2010.2019.csv")

In [ ]:
test.head()

In [ ]:
tmax   = pd.read_csv("../../../../../../../Volumes/backup2/dissData/prism/tempQuarterlyBins_2009.csv").drop(columns = {"Unnamed: 0"})
precip = pd.read_csv("../../../../../../../Volumes/backup2/dissData/prism/precipQuarterlyBins_2009.csv").drop(columns = {"Unnamed: 0"})

In [ ]:
allWeather = tmax.merge(precip)

In [ ]:
allWeather.head()

In [ ]:
allWeather['year'] = allWeather.yearQuarter.str.slice(0,4)
allWeather['qtr']  = allWeather.yearQuarter.str.slice(5,6).astype('float')
allWeather['zipcode']  = allWeather.zip.astype('int64')

In [ ]:
allWeather.zipcode.min()

In [ ]:
allWeather.to_csv("../../../../../../../Volumes/backup2/dissData/prism/allWeatherBins_2009.csv")